##Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/V&P/TLD

Mounted at /content/drive
/content/drive/MyDrive/V&P/TLD


##Libraries Import

In [ ]:
!pip install roboflow
!pip install albumentations==0.4.6
!pip install pytest-shutil

In [4]:
from roboflow import Roboflow
import torch
import glob as glob
import os
import pickle
import albumentations 
from albumentations.pytorch import ToTensorV2
import shutil

import numpy as np
import cv2
import torch
import glob as glob
import os
import time
from matplotlib import pyplot as plt
import albumentations as A

from model import create_model

from config import (
    NUM_CLASSES, DEVICE, CLASSES, MODEL_NAME, PRETRAINED
)

## Loading the Trained Model

In [201]:
model = create_model(num_classes=NUM_CLASSES, name=MODEL_NAME, pretrained=PRETRAINED)
checkpoint = torch.load('trainings/1/best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE).eval()

print('Model Loaded Correctly')

DIR_TEST = 'data/train'
test_images = glob.glob(f"{DIR_TEST}/*.jpg")
print(f"Test instances: {len(test_images)}")

Model Loaded Correctly
Test instances: 143


##Making the Predictions

In [205]:
THRESHOLD = 0.3
IMG_NUMBER = 2

In [206]:
def predict(img):

    # Processing to make the image net - compatible
    img = img.astype(np.float32)
    img /= 255.0
    img = np.transpose(img, (2, 0, 1)).astype(np.float32)

    img = torch.tensor(img, dtype=torch.float).cuda()

    #adding the batch dimension
    img = torch.unsqueeze(img, 0)

    with torch.no_grad():
        outputs = model(img.to(DEVICE))

    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]

    return outputs

## Display the predictions

In [207]:
def take_color(class_name): 
    if class_name == 'Red Car':
        color = (225, 0, 0)

    elif class_name == 'Red Ped': 
        color = (255, 0, 150)
    
    elif class_name == 'Green Car':
        color = (0, 255, 0)

    elif class_name == 'Green Ped': 
        color = (0, 255, 150)

    elif class_name == 'Yellow Car':
        color = (225, 150, 0)

    elif class_name == 'Yellow Ped': 
        color = (255, 150, 150)
    
    elif class_name == 'Unk': 
        color = (255, 0, 255)

    return color

In [208]:
def visualize_bbox(img, bbox, class_name):

    BOX_COLOR = take_color(class_name) # Red
    TEXT_COLOR = (255, 255, 255) # White

    x_min, x_max, y_min, y_max = bbox
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=BOX_COLOR, thickness=12)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 4, 2)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)

    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=4, 
        color=TEXT_COLOR,
        thickness = 10
    )
    return img


def visualize(image, threshold, outputs):

    if len(outputs[0]['boxes']) != 0:
        # getting the boxes
        boxes = outputs[0]['boxes'].data.numpy()
        # getting the scores
        scores = outputs[0]['scores'].data.numpy()

    # Bounding Boxes Predictions
    boxes = boxes[scores >= threshold].astype(np.int32)
    bboxes = boxes.copy()

    # Class Predictions
    pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()][:len(draw_boxes)]
    print(pred_classes)

    img = image.copy()
    for bbox, class_name in zip(bboxes, pred_classes):
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [210]:
for i in range(len(test_images[:10])):
    image = cv2.imread(test_images[i+40])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    outputs = predict(image)

    visualize(image, THRESHOLD, outputs)

Output hidden; open in https://colab.research.google.com to view.